# Addestra un LLM sui tuoi dati - Tutorial Davide De Rubeis
Questo notebook è ha lo scopo di addestrare un LLM open-source privato sui propri dati per poi esportarlo in GGFU per Ollama.

**Nota bene**: se vuoi usare la GPU gratuita vai da Modifica -> Impostazioni blocco note -> T4 GPU

Lo scopo è quello di creare un DataSet sulla sicurezza informatica e migliorare nelle seguenti categorie :

 - Cybersecuirty
 - Networking
 - Security , ecc..


**RINGRAZIAMENTI!!**

Progetto di partenza di Simone Rizzo a lui va i ringraziamenti e gli accrediti qui di seguito tutte le info necessarie :


 - **Come Addestrare un LLM con i Tuoi Dati Personali: Guida Completa con LLaMA 3.2**
https://youtu.be/UYOLlCuPFMc?si=i8WfU1-PZ7WL063A

 - **🔒 AI PRIVATA - LLM sul Tuo PC Senza Compromettere la Privacy! 🔒**
https://www.youtube.com/watch?v=EUOoNhxauNc

La demo utilizzata come partennza e il codice è stata realizzata da **Simone Rizzo**:
- [Youtube](https://www.youtube.com/channel/UCbMlkb79E12CwveGAtdFj-A)
- [Linkedin](https://www.linkedin.com/in/simone-rizzo-9851b7147/)
- [TikTok](https://www.tiktok.com/@simonerizzo98)
- [Instagram](https://www.instagram.com/simorizzo_ai/)

Seguilo e lascia un like sui sui social 😜

# Installazione delle librerie

Con il seguente comando carica il Driver di Google , con lo scopo di salvare i dati o caricare i dati.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Inizializzazione del modello

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.12.11: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "che cosa è lo shorinji kempo"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Lo Shorinji Kempo (also nota come Shotokan, Shorin Kempo, Shōrin-ryū, o Kempo) è un'arte marziale giapponese che combina elementi di Kyokushin Karate, Kobudo (arti marziali armate), Taekwondo e Karate. Lo Shorinji Kempo è stato sviluppato a metà del XX secolo dal maestro Kenwa Mabuni, che ha fondato il Gruppo Shorin Kempo.

**Origine**

Lo Shorinji K


# Formattazione del testo per il formato di LLama3.2

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
import pandas as pd

# Percorso del file Excel
file_excel = "/content/new_dataset.xlsx"

# Leggi l'Excel
df = pd.read_excel(file_excel)

# Salva come CSV
file_csv = "/content/new_dataset.csv"
df.to_csv(file_csv, index=False)

df = pd.read_csv("/content/new_dataset.csv")


In [ ]:
df.head()

,User,Prompt
0,Che cos'è Metasploit?,Metasploit è un framework open-source che vien...
1,Quali sono i moduli principali di Metasploit?,I principali moduli di Metasploit includono ex...
2,Cos'è un payload in Metasploit?,Un payload è un programma che viene eseguito s...
3,Che cos'è un exploit in Metasploit?,Un exploit è una porzione di codice che sfrutt...
4,Come si avvia Metasploit su Kali Linux?,"Per avviare Metasploit su Kali Linux, si utili..."


In [ ]:
df = df.dropna()

In [ ]:
df.columns

Index(['User', 'Prompt'], dtype='object')

In [ ]:
from datasets import Dataset

# Modifica il codice per utilizzare i nomi corretti delle colonne
df["conversations"] = df.apply(
    lambda x: [
        {"content": x["User"], "role": "user"},
        {"content": x["Prompt"], "role": "assistant"}
    ], axis=1
)

# Ora convertiamo il DataFrame in un Dataset di HuggingFace, rimuovendo le vecchie colonne
dataset = Dataset.from_pandas(df.drop(columns=["User", "Prompt"]))

print("Dataset HuggingFace creato con successo.")


Dataset HuggingFace creato con successo.


In [ ]:
dataset['conversations'][0]

[{'content': "Che cos'è Metasploit?", 'role': 'user'},
 {'content': 'Metasploit è un framework open-source che viene utilizzato per il testing di penetrazione e per sviluppare exploit contro vulnerabilità nei sistemi informatici.',
  'role': 'assistant'}]

# Addestramento del modello

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "conversations",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/224 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/224 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
13.07 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 224 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.565700
2,1.046100
3,0.722200
4,0.589100
5,0.872200
6,0.724900
7,0.657900
8,0.981000
9,0.830600
10,0.962700


config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

# Inferenza con Streaming

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "how can I configure Ruckus vlan 100 interface port 19"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

config vlan 100 interface Ethernet0/19<|eot_id|>


# Esporto in GGUF per Ollama

In [ ]:
model.save_pretrained_gguf("/content/model", tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
# model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# Save to 8bit Q8_0
# model.save_pretrained_gguf("model", tokenizer,)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.57 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 27.52it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /content/model/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at /content/model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weigh

In [ ]:
from google.colab import files
files.download('/content/model/unsloth.F16.gguf')

ALTRO

In [ ]:
!pip install transformers datasets accelerate peft


In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 51.2 MB/s eta 0:00:00
